# Template of a SMI jupyter notebook

In [ ]:
# These packages should all be installed if the procedure was followed
# %matplotlib notebook

import matplotlib.pyplot as plt
from smi_analysis import SMI_beamline
import numpy as np
import fabio
import os

## Experimental configuration

In [ ]:
geometry = 'Transmission'
energy = 2.450
wav = 1E-10 * (12.398/energy)
bs_kind = 'pindiode'
alphai = np.deg2rad(0)

#WAXS
detector_waxs = 'Pilatus1m'
sdd_waxs = 1800
center_waxs = [354, 560]
bs_pos_waxs = [[354, 535]]

SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                     detector = detector_waxs,
                                     sdd = sdd_waxs,
                                     wav = wav,
                                     alphai = alphai,
                                     center = center_waxs,
                                     bs_pos = bs_pos_waxs,
                                     bs_kind = bs_kind)


## Locate the files on your computer

In [ ]:
CYCLE='2024_1'
proposal_id =  '314903_Chaney_01'

path = '/nsls2/data/smi/legacy/results/data/%s/%s/1M/'%(CYCLE, proposal_id)

for file in os.listdir(path):
        print(file)

In [ ]:
#waxs
path = '/nsls2/data/smi/legacy/results/data/%s/%s/1M/'%(CYCLE, proposal_id)
ResDir = '/nsls2/data/smi/legacy/results/analysis/%s/%s/reduced_saxs/'%(CYCLE, proposal_id
                                                                       )

#waxs

sam, sam1 = [], []
for file in os.listdir(path):
     if 'wa20' in file and 'test' not in file:
        idx = file.find('_wa20')
        sam = sam + [file[:idx]]
        idx1=file.find('_loc')
        sam1 = sam1 + [file[idx1:idx1+6]]


print(sam)

all_dat = [[]] * len(sam)
all_da = [[]] * len(sam)
    
for j, (sa, sa1) in enumerate(zip(sam, sam1)): 
    for file in sorted(os.listdir(path)):
        if sa in file and sa1 in file and 'tif' in file:
            all_dat[j] = all_dat[j] + [file]
            
print(all_dat[0])

for i, all_d in enumerate(all_dat):
    for wa in ['wa20', 'wa40']:
        for dat in all_d:
            if wa in dat:
                all_da[i] = all_da[i] + [dat]
 
print(all_da[0])



In [ ]:
i=0
for dd in all_da:
    print(i, dd)
    i+=1
print(i)

## Data stitching and reduction (all this part is done using SMI_beamline package)

In [ ]:
all_da[0]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

#waxs
flag_save = True

for j, dat in enumerate([all_da[1]]):
        #This part is to stitch the data
        SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                             detector = detector_waxs,
                                             sdd = sdd_waxs,
                                             wav = wav,
                                             alphai = 0,
                                             center = center_waxs,
                                             bs_pos = bs_pos_waxs,
                                             det_angles = [0],
                                             bs_kind = bs_kind)
        
        
        print(dat)
        SMI_waxs.open_data(path, [dat[0]], optional_mask='tender')
        SMI_waxs.masks[0][560:, 337:350]=True
#         SMI_waxs.masks[0][954:, 337:350]=True

        plt.figure()
        for i, (img, mask) in enumerate(zip(SMI_waxs.imgs, SMI_waxs.masks)):
            SMI_waxs.imgs[i] *= 10
            plt.subplot(1, len(SMI_waxs.imgs), i+1)
            plt.imshow(img*np.logical_not(mask), vmin = 0, vmax = np.percentile(SMI_waxs.imgs[0], 99))
        plt.show()

        SMI_waxs.stitching_data(interp_factor=3)

        plt.figure()
        plt.imshow(SMI_waxs.img_st,
                   extent=[SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]], 
                   vmin=0,
                   vmax=np.percentile(SMI_waxs.img_st, 99))    
        plt.title('waxs_%s'%sam[j])
        plt.xlabel('$q_{par}(A^{-1})$')
        plt.ylabel('$q_{ver}(A^{-1})$')
        plt.colorbar()
        plt.show()

        SMI_waxs.caking()
        
        SMI_waxs.radial_averaging(radial_range = [0, 0.08], azimuth_range=[-179, 179], npt = 2000)
        q0, I0 = SMI_waxs.q_rad, SMI_waxs.I_rad

        SMI_waxs.radial_averaging(radial_range = [0, 0.08], azimuth_range=[-180, -90], npt = 2000)
        q1, I1 = SMI_waxs.q_rad, SMI_waxs.I_rad

#         SMI_waxs.radial_averaging(radial_range = [0, 0.08], azimuth_range=[-100, -80], npt = 2000)
#         q2, I2 = SMI_waxs.q_rad, SMI_waxs.I_rad
        
#         SMI_waxs.radial_averaging(radial_range = [0, 0.08], azimuth_range=[80, 100], npt = 2000)
#         q3, I3 = SMI_waxs.q_rad, SMI_waxs.I_rad

        plt.figure()
        plt.plot(q0, I0, linewidth = 3, color = 'k')
        plt.plot(q1, I1, linewidth = 3, color = 'g')
#         plt.plot(q2, I2, linewidth = 3, color = 'r')
#         plt.plot(q3, I3, linewidth = 3, color = 'b')
#         plt.plot(q4, I4, linewidth = 3, color = 'c')

        plt.yscale('log')
#         plt.xscale('log')
        plt.show()

plt.close('all')
print('Done')

In [ ]:
plt.imshow(SMI_waxs.cake, norm=plt.Normalize(1e8,5e9))
plt.colorbar()
plt.show()

In [ ]:
0.1*2*np.pi/0.025

In [ ]:

#waxs
flag_save = True

for j, dat in enumerate(all_da):          
        alphai = np.deg2rad(0)
        waxs_angle = [np.deg2rad(0)]
        
        if len(dat)==1:# and 'q_map_waxs_%s%s.png'%(sam[j], sam1[j]) not in os.listdir(ResDir):

        
            #This part is to stitch the data
            SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                                 detector = detector_waxs,
                                                 sdd = sdd_waxs,
                                                 wav = wav,
                                                 alphai = alphai,
                                                 center = center_waxs,
                                                 bs_pos = bs_pos_waxs,
                                                 det_angles = waxs_angle,
                                                 bs_kind = bs_kind)

            SMI_waxs.open_data(path, [dat[0]])
            
            for i, da in enumerate(dat):
                if i!=0:
                    img=fabio.open(os.path.join(path, da)).data
                    SMI_waxs.imgs[0]+=img

            SMI_waxs.masks[0][560:, 475:482]=True
            SMI_waxs.masks[0][954:, 767:769]=True
        
            SMI_waxs.stitching_data(interp_factor=3)

            plt.figure()
            plt.imshow(SMI_waxs.img_st,
                       extent=[SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]], 
                       vmin=0,
                       vmax=np.percentile(SMI_waxs.img_st, 99.5))    
            plt.title('waxs_%s%s'%(sam[j], sam1[j]))
            plt.xlabel('$q_{par}(A^{-1})$')
            plt.ylabel('$q_{ver}(A^{-1})$')
            plt.colorbar()
            plt.show()

            SMI_waxs.radial_averaging(radial_range = [0, 0.21], azimuth_range=[-30, 30], npt = 2000)
            q0, I0 = SMI_waxs.q_rad, SMI_waxs.I_rad
            
            SMI_waxs.radial_averaging(radial_range = [0, 0.21], azimuth_range=[-120, -60], npt = 2000)
            q1, I1 = SMI_waxs.q_rad, SMI_waxs.I_rad
            
            SMI_waxs.radial_averaging(radial_range = [0, 0.21], azimuth_range=[-179, 179], npt = 2000)
            q2, I2 = SMI_waxs.q_rad, SMI_waxs.I_rad
            
            
            if flag_save:
                #Save the plot as a png
                plt.savefig(os.path.join(ResDir, 'q_map_waxs_%s%s.png'%(sam[j], sam1[j])))

                np.savetxt(os.path.join(ResDir, 'Radint_hor_%s%s.txt'%(sam[j], sam1[j])), np.vstack((q0, I0)).T)
                np.savetxt(os.path.join(ResDir, 'Radint_ver_%s%s.txt'%(sam[j], sam1[j])), np.vstack((q1, I1)).T)
                np.savetxt(os.path.join(ResDir, 'Radint_tot_%s%s.txt'%(sam[j], sam1[j])), np.vstack((q2, I2)).T)

            plt.close() 
            
        else:
            print('already done')
print('Done')